This is a temporary set of tests.

In [1]:
from quantumgraph import QuantumGraph, ExpectationValue
import numpy as np

An IBMQ account could not be loaded


Set a tolerance for how close the numbers generated by `QuantumGraph` should be to the correct values.

In [2]:
eps = 0.5  # Tolerance for numerical errors

Whether to test using expectation value simulator.

In [3]:
test_ev = False

Tests are done on a graph of 5 qubits

In [4]:
n = 5

if test_ev:
    device = ExpectationValue(n)
    graph = QuantumGraph(n,device=device)
else:
    graph = QuantumGraph(n)

First we check that all qubits are indeed in the $\langle Z \rangle = 1$ state.

In [5]:
initial_rho = {'X': 0.0, 'Y': 0.0, 'Z': 1.0}
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)

Now set qubit 2 to the $\langle X \rangle = 1$ state, check that it works and also that the rest are still in the $\langle Z \rangle = 1$ state.

In [6]:
rhos = {}
rho_2 = {'X': 1.0}
graph.set_bloch(rho_2,2)
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        else:
            assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)

The relationship between qubit 2 and any other should now have $\langle ZX \rangle = 1$ (or $\langle XZ \rangle = 1$ depending on order. Check that this is true.

In [7]:
for j in range(n):
    if j!=2:
        assert abs(graph.get_relationship(2,j)['XZ']-1)<eps
        assert abs(graph.get_relationship(j,2)['ZX']-1)<eps
# for j in range(n):
#     if j != 2:
#         bloch2 = graph.get_bloch(2)
#         blochj = graph.get_bloch(j)
#         rel = graph.get_relationship(2, j)
#         expected = bloch2['X'] * blochj['Z']
#         print(f"Relationship (2,{j}):", rel)
#         assert abs(rel.get('XZ', 0.0) - expected) < eps

Set qubit 1 to the $\langle Y \rangle=1$ state (without updating the tomography), and then set qubit 3 to the $\langle X \rangle = \langle Z \rangle$ and $\langle Z \rangle=0$ state (with the update). Check that they are correct, and that the others are still what they should be too.

In [ ]:
rho_1 = {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
rho_3 = {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
graph.set_bloch(rho_1, 1, update=False)
graph.set_bloch(rho_3, 3)

rho_3 = {'X': 1/np.sqrt(2), 'Y': 1/np.sqrt(2), 'Z': 0.0}
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==1:
            assert abs(rho[pauli]-rho_1[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==3:
            assert abs(rho[pauli]-rho_3[pauli])<eps, print('Problem with qubit',j,':',rho)
        else:
            assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)

Problem with qubit 1 : {'X': np.float64(0.7021296316373856), 'Y': np.float64(0.7036283780718682), 'Z': np.float64(-0.005281047152389842)}


AssertionError: None

Do half of the rotation of qubit 4 from the $\langle Z \rangle=1$ state to $\langle X \rangle=-1$.

In [23]:
rho_4 = {'X': -1.0, 'Y': 0.0, 'Z': 0.0}
graph.set_bloch(rho_4,4,fraction=0.5)
rho_4 = {'X': -1/np.sqrt(2), 'Y': 0.0, 'Z': 1/np.sqrt(2)}
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==1:
            assert abs(rho[pauli]-rho_1[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==3:
            assert abs(rho[pauli]-rho_3[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==4:
            assert abs(rho[pauli]-rho_4[pauli])<eps, print('Problem with qubit',j,':',rho)

Problem with qubit 1 : {'X': np.float64(-0.366847507904458), 'Y': np.float64(0.2742924857039773), 'Z': np.float64(-0.8807910213857473)}


AssertionError: None

Various tests for two qubits.

In [24]:
relationships = {'ZZ':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)

relationships = {'XX':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)
    
relationships = {'ZX':+1,'XZ':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)

Problem with `set_relationship`: {'XX': np.float64(0.0028741177365946995), 'XY': np.float64(0.0008282585887333046), 'XZ': np.float64(0.32741859683609403), 'YX': np.float64(0.001910279522017914), 'YY': np.float64(-6.187697804163877e-05), 'YZ': np.float64(-0.23952077894592877), 'ZX': np.float64(0.0013282809836104672), 'ZY': np.float64(-0.004779353810466715), 'ZZ': np.float64(0.7695974134269598)}


AssertionError: None